In [ ]:
# !sudo python3 -m pip install -U slackclient

In [3]:
# !sudo apt-get install python3-mysqldb

In [33]:
%matplotlib inline

In [1]:
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
import json
from slackclient import SlackClient

# Read the access token from the file
secrets_file = 'slack_secret.json'
f = open(secrets_file, 'r') 
content = f.read()
f.close()
auth_info = json.loads(content)
token = auth_info["access_token"]
sc = SlackClient(token)


In [22]:
# Parameters to connect to MySQL database
parameters = {
    'host':     'localhost', 
    'username': 'root', 
    'password': 'dwdstudent2015', 
    'database': 'slack',
    'port':     '3306',
    'charset':  'utf8',
    'use_unicode': True
}

In [14]:
import MySQLdb as mdb
con = mdb.connect(host = parameters['host'], 
                  user = parameters['username'], 
                  passwd = parameters['password'], 
                  charset = parameters['charset'],
                  use_unicode = parameters['use_unicode']);

In [66]:
# Create the database
create_db_query = "CREATE DATABASE IF NOT EXISTS {0} DEFAULT CHARACTER SET 'utf8mb4'".format(parameters['database'])
cursor = con.cursor()
cursor.execute(create_db_query)
cursor.close()
con.close()

/usr/lib/python3/dist-packages/IPython/kernel/__main__.py:4: Warning: Can't create database 'slack_detectica'; database exists


In [67]:
import mysql.connector
from sqlalchemy import create_engine


template = 'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}?charset={charset}'
conn_string = template.format(**parameters)

engine = create_engine(conn_string, echo=False)


In [26]:
# Get channel list and store in database
response = sc.api_call("channels.list")
df_channels = json_normalize(response["channels"])
df_channels.drop('members', axis=1, inplace = True)
df_channels.to_sql(name='channels', con=engine, if_exists='replace')
#df_channels

In [27]:
# Get users list and store in database
response = sc.api_call("users.list")
response
df_users = json_normalize(response["members"])
df_users.to_sql(name='users', con=engine, if_exists='replace')
#df_users

In [ ]:
# Retrieves all past messages from all public channels
# and stores them in the database
#
# TODO: Modify to first query for latest message stored in database
# for each channel, and fetch/store only the later ones

import sys

def get_messages(channel_id, latest_fetched_ts): 
    oldest = sys.maxsize
    results = []
    while True:
        response = sc.api_call("channels.history", channel=channel_id, latest=oldest)
        data = [{"timestamp": entry.get("ts"), "user": entry.get("user"), "message": entry.get("text")} for entry in response["messages"] ]
        results.extend(data)
        oldest = min([entry["ts"] for entry in response["messages"]])
        # oldest_ts = int(round(float(oldest)))
        latest = max([entry["ts"] for entry in response["messages"]])
        # latest_ts = int(round(float(latest)))
        
        
        if response["has_more"] == False:
            break
        
    df = pd.DataFrame(results)
    return df

for i, row in df_channels[["id", "name"]].iterrows():
    
    channel_id = row["id"]
    channel_name = row["name"]
    print(channel_id, channel_name)
    df_msgs = get_messages(channel_id, sys.maxsize)
    df_msgs["channel_id"] = row["id"]
    df_msgs.to_sql(name='messages', con=engine, if_exists='append')


In [28]:
# Read all the stored messages in a Pandas Dataframe
df_msgs = pd.read_sql("SELECT * FROM messages", con=engine)
df_msgs['timestamp'] = pd.to_numeric(df_msgs['timestamp'])
df_msgs['timestamp'] = pd.to_datetime(df_msgs['timestamp'], unit='s')
df_msgs.set_index('timestamp', inplace=True)
df_msgs.sort_index(inplace=True)
df_msgs.drop(['index'], axis=1, inplace=True)

In [31]:
# Function to do basic sentiment analysis on the Slack messages
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def sentiment_apply(x):
    sid = SentimentIntensityAnalyzer()
    if (not pd.isnull(x['message'])):
        return pd.Series(sid.polarity_scores(x['message'])) 
    else: 
        return pd.Series({'pos': None, 'neu': None, 'compound': None, 'neg': None})

In [32]:
# Run sentiment analysis on all messages
# and store results in database
newcols = df_msgs.apply(sentiment_apply, axis=1)
newcols.columns = ['pos', 'neu', 'compound', 'neg']
df_msgs = df_msgs.join(newcols)
df_msgs.to_sql(name='messages', con=engine, if_exists='replace')

,message,user,channel_id,pos,neu,compound,neg
timestamp,,,,,,,
2015-01-15 03:22:18,<@U03C9Q002|panos> has joined the channel,U03C9Q002,C03C9Q00W,0.0000,0.000,1.000,0.000
2015-01-15 03:22:18,<@U03C9Q002|panos> has joined the channel,U03C9Q002,C03C9Q00W,0.0000,0.000,1.000,0.000
2015-01-15 03:22:18,<@U03C9Q002|panos> has joined the channel,U03C9Q002,C03C9Q00U,0.0000,0.000,1.000,0.000
2015-01-15 03:22:18,<@U03C9Q002|panos> has joined the channel,U03C9Q002,C03C9Q00U,0.0000,0.000,1.000,0.000
2015-01-15 06:39:23,<@U03C8NS83|jattenberg> has joined the channel,U03C8NS83,C03C9Q00W,0.0000,0.000,1.000,0.000
2015-01-15 06:39:23,<@U03C8NS83|jattenberg> has joined the channel,U03C8NS83,C03C9Q00W,0.0000,0.000,1.000,0.000
2015-01-15 06:39:23,<@U03C8NS83|jattenberg> has joined the channel,U03C8NS83,C03C9Q00U,0.0000,0.000,1.000,0.000
2015-01-15 06:39:23,<@U03C8NS83|jattenberg> has joined the channel,U03C8NS83,C03C9Q00U,0.0000,0.000,1.000,0.000
2015-01-15 06:40:25,hi,U03C8NS83,C03C9Q00U,0.0000,0.000,1.000,0.000
